In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5.nc')
print(ds)
ivt_concat_era5_variable=ds['ivt_concat_era5_variable']
iwv_concat_era5_variable=ds['iwv_concat_era5_variable']
s_concat_era5_variable=ds['s_concat_era5_variable']
mfc_concat_era5_variable=ds['mfc_concat_era5_variable']

ivt_concat_era5_fixed=ds['ivt_concat_era5_fixed']
iwv_concat_era5_fixed=ds['iwv_concat_era5_fixed']
s_concat_era5_fixed=ds['s_concat_era5_fixed']
mfc_concat_era5_fixed=ds['mfc_concat_era5_fixed']

ivt_concat_era5_fixed_poleward=ds['ivt_concat_era5_fixed_poleward']
iwv_concat_era5_fixed_poleward=ds['iwv_concat_era5_fixed_poleward']
s_concat_era5_fixed_poleward=ds['s_concat_era5_fixed_poleward']
mfc_concat_era5_fixed_poleward=ds['mfc_concat_era5_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_merra2.nc')
print(ds)
ivt_concat_merra2_variable=ds['ivt_concat_merra2_variable']
iwv_concat_merra2_variable=ds['iwv_concat_merra2_variable']
s_concat_merra2_variable=ds['s_concat_merra2_variable']
mfc_concat_merra2_variable=ds['mfc_concat_merra2_variable']

ivt_concat_merra2_fixed=ds['ivt_concat_merra2_fixed']
iwv_concat_merra2_fixed=ds['iwv_concat_merra2_fixed']
s_concat_merra2_fixed=ds['s_concat_merra2_fixed']
mfc_concat_merra2_fixed=ds['mfc_concat_merra2_fixed']

ivt_concat_merra2_fixed_poleward=ds['ivt_concat_merra2_fixed_poleward']
iwv_concat_merra2_fixed_poleward=ds['iwv_concat_merra2_fixed_poleward']
s_concat_merra2_fixed_poleward=ds['s_concat_merra2_fixed_poleward']
mfc_concat_merra2_fixed_poleward=ds['mfc_concat_merra2_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_jra55.nc')
print(ds)
ivt_concat_jra55_variable=ds['ivt_concat_jra55_variable']
iwv_concat_jra55_variable=ds['iwv_concat_jra55_variable']
s_concat_jra55_variable=ds['s_concat_jra55_variable']

ivt_concat_jra55_fixed=ds['ivt_concat_jra55_fixed']
iwv_concat_jra55_fixed=ds['iwv_concat_jra55_fixed']
s_concat_jra55_fixed=ds['s_concat_jra55_fixed']

ivt_concat_jra55_fixed_poleward=ds['ivt_concat_jra55_fixed_poleward']
iwv_concat_jra55_fixed_poleward=ds['iwv_concat_jra55_fixed_poleward']
s_concat_jra55_fixed_poleward=ds['s_concat_jra55_fixed_poleward']

In [ ]:
fieldc=ivt_concat_era5_fixed.mean(dim='time',skipna=True)
fielda=ivt_concat_merra2_fixed.mean(dim='time',skipna=True)
fieldb=ivt_concat_jra55_fixed.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed/ivt_concat_era5_fixed
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed/ivt_concat_merra2_fixed
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed/ivt_concat_jra55_fixed
field1b=field1b.interp_like(ivt_concat_merra2_fixed)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=600,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(100,700,100))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,660,60),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(60,660,120))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IVT}$'+' '+r'$\bf{fixed}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_1.png')

In [ ]:
fieldc=ivt_concat_era5_fixed_poleward.mean(dim='time',skipna=True)
fielda=ivt_concat_merra2_fixed_poleward.mean(dim='time',skipna=True)
fieldb=ivt_concat_jra55_fixed_poleward.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed_poleward/ivt_concat_era5_fixed_poleward
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed_poleward/ivt_concat_merra2_fixed_poleward
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed_poleward/ivt_concat_jra55_fixed_poleward
field1b=field1b.interp_like(ivt_concat_merra2_fixed_poleward)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=600,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(100,700,100))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,660,60),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(60,660,120))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IVT}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_2.png')

In [ ]:
fieldc=ivt_concat_era5_variable.mean(dim='time',skipna=True)
fielda=ivt_concat_merra2_variable.mean(dim='time',skipna=True)
fieldb=ivt_concat_jra55_variable.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_variable/ivt_concat_era5_variable
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_variable/ivt_concat_merra2_variable
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_variable/ivt_concat_jra55_variable
field1b=field1b.interp_like(ivt_concat_merra2_variable)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=600,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(100,700,100))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,660,60),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(60,660,120))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IVT}$'+' '+r'$\bf{variable}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_3.png')

In [ ]:
fieldc=iwv_concat_era5_fixed.mean(dim='time',skipna=True)
fielda=iwv_concat_merra2_fixed.mean(dim='time',skipna=True)
fieldb=iwv_concat_jra55_fixed.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed/ivt_concat_era5_fixed
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed/ivt_concat_merra2_fixed
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed/ivt_concat_jra55_fixed
field1b=field1b.interp_like(ivt_concat_merra2_fixed)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=70,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,64,8),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(8,64,8))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-2}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{fixed}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_4.png')

In [ ]:
fieldc=iwv_concat_era5_fixed_poleward.mean(dim='time',skipna=True)
fielda=iwv_concat_merra2_fixed_poleward.mean(dim='time',skipna=True)
fieldb=iwv_concat_jra55_fixed_poleward.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed_poleward/ivt_concat_era5_fixed_poleward
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed_poleward/ivt_concat_merra2_fixed_poleward
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed_poleward/ivt_concat_jra55_fixed_poleward
field1b=field1b.interp_like(ivt_concat_merra2_fixed_poleward)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=50,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(10,60,10))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,64,8),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(8,64,8))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-2}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_5.png')

In [ ]:
fieldc=iwv_concat_era5_variable.mean(dim='time',skipna=True)
fielda=iwv_concat_merra2_variable.mean(dim='time',skipna=True)
fieldb=iwv_concat_jra55_variable.mean(dim='time',skipna=True)
fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_variable/ivt_concat_era5_variable
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_variable/ivt_concat_merra2_variable
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_variable/ivt_concat_jra55_variable
field1b=field1b.interp_like(ivt_concat_merra2_variable)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=50,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(10,60,10))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,64,8),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(8,64,8))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('kg m$^{-2}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{variable}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_6.png')

In [ ]:
fieldc=s_concat_era5_fixed.mean(dim='time',skipna=True)
fielda=s_concat_merra2_fixed.mean(dim='time',skipna=True)
fieldb=s_concat_jra55_fixed.mean(dim='time',skipna=True)

dk=xr.Dataset()
dk['test']=(('lat','lon'),fieldb.values)
dk.coords['lat']=fieldb.lat1.values
dk.coords['lon']=fieldb.lon1.values
fieldb=dk['test']

fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed/ivt_concat_era5_fixed
field1=field1.sum(dim='time')

field1a=ivt_concat_merra2_fixed/ivt_concat_merra2_fixed
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed/ivt_concat_jra55_fixed

dk=xr.Dataset()
dk['test']=(('time','lat','lon'),field1b.values)
dk.coords['time']=field1b.time.values
dk.coords['lat']=field1b.lat.values
dk.coords['lon']=field1b.lon.values
field1b=dk['test']


field1b=field1b.interp_like(ivt_concat_merra2_fixed)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=25,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(5,30,5))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,30,3),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(3,30,6))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('m s$^{-1}$',fontsize=55,labelpad=15)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$'+' '+r'$\bf{fixed}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_7.png')

In [ ]:
fieldc=s_concat_era5_fixed_poleward.mean(dim='time',skipna=True)
fielda=s_concat_merra2_fixed_poleward.mean(dim='time',skipna=True)
fieldb=s_concat_jra55_fixed_poleward.mean(dim='time',skipna=True)

dk=xr.Dataset()
dk['test']=(('lat','lon'),fieldb.values)
dk.coords['lat']=fieldb.lat1.values
dk.coords['lon']=fieldb.lon1.values
fieldb=dk['test']

fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=ivt_concat_era5_fixed_poleward/ivt_concat_era5_fixed_poleward
field1=field1.sum(dim='time')

field1a=ivt_concat_merra2_fixed_poleward/ivt_concat_merra2_fixed_poleward
field1a=field1a.sum(dim='time')

field1b=ivt_concat_jra55_fixed_poleward/ivt_concat_jra55_fixed_poleward

dk=xr.Dataset()
dk['test']=(('time','lat','lon'),field1b.values)
dk.coords['time']=field1b.time.values
dk.coords['lat']=field1b.lat.values
dk.coords['lon']=field1b.lon.values
field1b=dk['test']


field1b=field1b.interp_like(ivt_concat_merra2_fixed_poleward)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=25,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(5,30,5))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,30,3),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(3,30,6))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('m s$^{-1}$',fontsize=55,labelpad=15)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=60,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_8.png')

In [ ]:
fieldc=s_concat_era5_variable.mean(dim='time',skipna=True)
fielda=s_concat_merra2_variable.mean(dim='time',skipna=True)
fieldb=s_concat_jra55_variable.mean(dim='time',skipna=True)

dk=xr.Dataset()
dk['test']=(('lat','lon'),fieldb.values)
dk.coords['lat']=fieldb.lat1.values
dk.coords['lon']=fieldb.lon1.values
fieldb=dk['test']

fieldb=fieldb.interp_like(fielda)
field=(fielda+fieldb+fieldc)/3.

field1=iwv_concat_era5_variable/iwv_concat_era5_variable
field1=field1.sum(dim='time')

field1a=iwv_concat_merra2_variable/iwv_concat_merra2_variable
field1a=field1a.sum(dim='time')

field1b=iwv_concat_jra55_variable/iwv_concat_jra55_variable

dk=xr.Dataset()
dk['test']=(('time','lat','lon'),field1b.values)
dk.coords['time']=field1b.time.values
dk.coords['lat']=field1b.lat.values
dk.coords['lon']=field1b.lon.values
field1b=dk['test']


field1b=field1b.interp_like(ivt_concat_merra2_variable)
field1b=field1b.sum(dim='time')

field_freq=(field1+field1a+field1b)/3.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=25,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(5,30,5))
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,30,3),extend='max',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(3,30,6))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('m s$^{-1}$',fontsize=55,labelpad=15)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$'+' '+r'$\bf{variable}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_9.png')

In [ ]:
fieldc=mfc_concat_era5_fixed.mean(dim='time',skipna=True)
fielda=mfc_concat_merra2_fixed.mean(dim='time',skipna=True)
field=(fielda+fieldc)/2./1e-4

field1=ivt_concat_era5_fixed/ivt_concat_era5_fixed
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed/ivt_concat_merra2_fixed
field1a=field1a.sum(dim='time')

field_freq=(field1+field1a)/2.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.4,vmax=1.4,cmap=plt.cm.BrBG,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15,ticks=[-1,-0.5,0,0.5,1])
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,2.4,0.3),extend='both',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('10$^{-4}$ kg m$^{-2}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{MFC}$'+' '+r'$\bf{fixed}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_10.png')

In [ ]:
fieldc=mfc_concat_era5_fixed_poleward.mean(dim='time',skipna=True)
fielda=mfc_concat_merra2_fixed_poleward.mean(dim='time',skipna=True)
field=(fielda+fieldc)/2./1e-4

field1=ivt_concat_era5_fixed_poleward/ivt_concat_era5_fixed_poleward
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_fixed_poleward/ivt_concat_merra2_fixed_poleward
field1a=field1a.sum(dim='time')

field_freq=(field1+field1a)/2.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.4,vmax=1.4,cmap=plt.cm.BrBG,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15,ticks=[-1,-0.5,0,0.5,1])
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,2.4,0.3),extend='both',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('10$^{-4}$ kg m$^{-2}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{MFC}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_11.png')

In [ ]:
fieldc=mfc_concat_era5_variable.mean(dim='time',skipna=True)
fielda=mfc_concat_merra2_variable.mean(dim='time',skipna=True)
field=(fielda+fieldc)/2./1e-4

field1=ivt_concat_era5_variable/ivt_concat_era5_variable
field1=field1.sum(dim='time').values

field1a=ivt_concat_merra2_variable/ivt_concat_merra2_variable
field1a=field1a.sum(dim='time')

field_freq=(field1+field1a)/2.

field=field.where(field_freq>=0.5*44)

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.4,vmax=1.4,cmap=plt.cm.BrBG,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15,ticks=[-1,-0.5,0,0.5,1])
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(0,2.4,0.3),extend='both',cmap=plt.cm.viridis,alpha=1.0)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('10$^{-4}$ kg m$^{-2}$ s$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{MFC}$'+' '+r'$\bf{variable}$',fontsize=64,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_vars_geomeans_12.png')